In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
games = pd.read_csv('data/MasterGames_2000to2019.csv')
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,Hteam,HQ1,HQ2,HQ3,HQ4,Date,Time,Winner/tie,Unnamed: 18,Loser/tie
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,Vikings,6,3,7,14,9/3/2000,1:00PM,Minnesota Vikings,NaN,Chicago Bears
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,Chiefs,0,7,7,0,9/3/2000,1:00PM,Indianapolis Colts,@,Kansas City Chiefs
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,Redskins,7,0,3,10,9/3/2000,1:01PM,Washington Redskins,NaN,Carolina Panthers
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,Falcons,6,16,14,0,9/3/2000,1:02PM,Atlanta Falcons,NaN,San Francisco 49ers
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,Steelers,0,0,0,0,9/3/2000,1:02PM,Baltimore Ravens,@,Pittsburgh Steelers


In [3]:
games = games.drop(['Winner/tie','Unnamed: 18','Loser/tie'],axis=1)
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,Hteam,HQ1,HQ2,HQ3,HQ4,Date,Time
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,Vikings,6,3,7,14,9/3/2000,1:00PM
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,Chiefs,0,7,7,0,9/3/2000,1:00PM
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,Redskins,7,0,3,10,9/3/2000,1:01PM
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,Falcons,6,16,14,0,9/3/2000,1:02PM
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,Steelers,0,0,0,0,9/3/2000,1:02PM


In [4]:
games['RQ1'] = games['VQ1'] - games['HQ1']
games['RQ2'] = games['VQ2'] - games['HQ2']
games['RQ3'] = games['VQ3'] - games['HQ3']
games['RQ4'] = games['VQ4'] - games['HQ4']
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ1,HQ2,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,6,3,7,14,9/3/2000,1:00PM,1,3,0,-7
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,0,7,7,0,9/3/2000,1:00PM,0,0,0,13
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,7,0,3,10,9/3/2000,1:01PM,3,0,-3,-3
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,6,16,14,0,9/3/2000,1:02PM,1,-9,-7,7
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,0,0,9/3/2000,1:02PM,10,3,3,0


In [5]:
games['Win'] = games['RQ1']+games['RQ2']+games['RQ3']+games['RQ4']
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ2,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4,Win
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,3,7,14,9/3/2000,1:00PM,1,3,0,-7,-3
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,7,7,0,9/3/2000,1:00PM,0,0,0,13,13
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,0,3,10,9/3/2000,1:01PM,3,0,-3,-3,-3
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,16,14,0,9/3/2000,1:02PM,1,-9,-7,7,-8
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,0,9/3/2000,1:02PM,10,3,3,0,16


In [6]:
games['Winner'] = 'X'
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4,Win,Winner
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,7,14,9/3/2000,1:00PM,1,3,0,-7,-3,X
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,7,0,9/3/2000,1:00PM,0,0,0,13,13,X
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,3,10,9/3/2000,1:01PM,3,0,-3,-3,-3,X
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,14,0,9/3/2000,1:02PM,1,-9,-7,7,-8,X
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,9/3/2000,1:02PM,10,3,3,0,16,X


In [7]:
for win in range(0,games.shape[0]):
    if games.iloc[win,-2] > 0:
        games.iloc[win,-1] = 'VW'
    elif games.iloc[win,-2] < 0:
        games.iloc[win,-1] = 'HW'
    else:
        games.iloc[win,-1] = 'T'
games.head()


,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4,Win,Winner
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,7,14,9/3/2000,1:00PM,1,3,0,-7,-3,HW
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,7,0,9/3/2000,1:00PM,0,0,0,13,13,VW
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,3,10,9/3/2000,1:01PM,3,0,-3,-3,-3,HW
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,14,0,9/3/2000,1:02PM,1,-9,-7,7,-8,HW
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,9/3/2000,1:02PM,10,3,3,0,16,VW


In [8]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5324 entries, 0 to 5323
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   GameID     5324 non-null   object
 1   Week       5324 non-null   int64 
 2   Day        5324 non-null   object
 3   DateStamp  5324 non-null   object
 4   Vteam      5324 non-null   object
 5   VQ1        5324 non-null   int64 
 6   VQ2        5324 non-null   int64 
 7   VQ3        5324 non-null   int64 
 8   VQ4        5324 non-null   int64 
 9   Year       5324 non-null   int64 
 10  Hteam      5324 non-null   object
 11  HQ1        5324 non-null   int64 
 12  HQ2        5324 non-null   int64 
 13  HQ3        5324 non-null   int64 
 14  HQ4        5324 non-null   int64 
 15  Date       5324 non-null   object
 16  Time       5324 non-null   object
 17  RQ1        5324 non-null   int64 
 18  RQ2        5324 non-null   int64 
 19  RQ3        5324 non-null   int64 
 20  RQ4        5324 non-null   int

In [9]:
for win in range(0,games.shape[0]):
    if games.iloc[win,17] < -14:
        games.iloc[win,17] = 'HHH'
    elif games.iloc[win,17] < -7:
        games.iloc[win,17] = 'HH'
    elif games.iloc[win,17] < 0:
        games.iloc[win,17] = 'H'
    elif games.iloc[win,17] == 0:
        games.iloc[win,17] = 'T'
    elif games.iloc[win,17] < 8:
        games.iloc[win,17] = 'V'
    elif games.iloc[win,17] < 15:
        games.iloc[win,17] = 'VV'
    elif games.iloc[win,17] > 14:
        games.iloc[win,17] = 'VVV'
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4,Win,Winner
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,7,14,9/3/2000,1:00PM,V,3,0,-7,-3,HW
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,7,0,9/3/2000,1:00PM,T,0,0,13,13,VW
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,3,10,9/3/2000,1:01PM,V,0,-3,-3,-3,HW
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,14,0,9/3/2000,1:02PM,V,-9,-7,7,-8,HW
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,9/3/2000,1:02PM,VV,3,3,0,16,VW


In [10]:
for win in range(0,games.shape[0]):
    if games.iloc[win,18] < -14:
        games.iloc[win,18] = 'HHH'
    elif games.iloc[win,18] < -7:
        games.iloc[win,18] = 'HH'
    elif games.iloc[win,18] < 0:
        games.iloc[win,18] = 'H'
    elif games.iloc[win,18] == 0:
        games.iloc[win,18] = 'T'
    elif games.iloc[win,18] < 8:
        games.iloc[win,18] = 'V'
    elif games.iloc[win,18] < 15:
        games.iloc[win,18] = 'VV'
    elif games.iloc[win,18] > 14:
        games.iloc[win,18] = 'VVV'
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4,Win,Winner
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,7,14,9/3/2000,1:00PM,V,V,0,-7,-3,HW
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,7,0,9/3/2000,1:00PM,T,T,0,13,13,VW
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,3,10,9/3/2000,1:01PM,V,T,-3,-3,-3,HW
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,14,0,9/3/2000,1:02PM,V,HH,-7,7,-8,HW
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,9/3/2000,1:02PM,VV,V,3,0,16,VW


In [11]:
for win in range(0,games.shape[0]):
    if games.iloc[win,19] < -14:
        games.iloc[win,19] = 'HHH'
    elif games.iloc[win,19] < -7:
        games.iloc[win,19] = 'HH'
    elif games.iloc[win,19] < 0:
        games.iloc[win,19] = 'H'
    elif games.iloc[win,19] == 0:
        games.iloc[win,19] = 'T'
    elif games.iloc[win,19] < 8:
        games.iloc[win,19] = 'V'
    elif games.iloc[win,19] < 15:
        games.iloc[win,19] = 'VV'
    elif games.iloc[win,19] > 14:
        games.iloc[win,19] = 'VVV'
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4,Win,Winner
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,7,14,9/3/2000,1:00PM,V,V,T,-7,-3,HW
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,7,0,9/3/2000,1:00PM,T,T,T,13,13,VW
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,3,10,9/3/2000,1:01PM,V,T,H,-3,-3,HW
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,14,0,9/3/2000,1:02PM,V,HH,H,7,-8,HW
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,9/3/2000,1:02PM,VV,V,V,0,16,VW


In [12]:
for win in range(0,games.shape[0]):
    if games.iloc[win,20] < -14:
        games.iloc[win,20] = 'HHH'
    elif games.iloc[win,20] < -7:
        games.iloc[win,20] = 'HH'
    elif games.iloc[win,20] < 0:
        games.iloc[win,20] = 'H'
    elif games.iloc[win,20] == 0:
        games.iloc[win,20] = 'T'
    elif games.iloc[win,20] < 8:
        games.iloc[win,20] = 'V'
    elif games.iloc[win,20] < 15:
        games.iloc[win,20] = 'VV'
    elif games.iloc[win,20] > 14:
        games.iloc[win,20] = 'VVV'
games.head()

,GameID,Week,Day,DateStamp,Vteam,VQ1,VQ2,VQ3,VQ4,Year,...,HQ3,HQ4,Date,Time,RQ1,RQ2,RQ3,RQ4,Win,Winner
0,200009030min,1,Sun,2000-09-03,Bears,7,6,7,7,2000,...,7,14,9/3/2000,1:00PM,V,V,T,H,-3,HW
1,200009030kan,1,Sun,2000-09-03,Colts,0,7,7,13,2000,...,7,0,9/3/2000,1:00PM,T,T,T,VV,13,VW
2,200009030was,1,Sun,2000-09-03,Panthers,10,0,0,7,2000,...,3,10,9/3/2000,1:01PM,V,T,H,H,-3,HW
3,200009030atl,1,Sun,2000-09-03,49ers,7,7,7,7,2000,...,14,0,9/3/2000,1:02PM,V,HH,H,V,-8,HW
4,200009030pit,1,Sun,2000-09-03,Ravens,10,3,3,0,2000,...,0,0,9/3/2000,1:02PM,VV,V,V,T,16,VW


In [13]:
#save data
games.to_csv('data/Quarterly_Wins.csv')
